# Data Science Final Project

In [1]:
# !pip install nltk
# !pip install keras
# !pip install tensorflow
from matplotlib import pyplot
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

from datetime import datetime
import time
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import gc
from dateutil import parser
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer


plt.rcParams.update(plt.rcParamsDefault)
%matplotlib inline
pd.set_option('display.max_columns', None)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tdepa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


https://www.kaggle.com/datasets/kazanova/sentiment140

https://www.analyticsvidhya.com/blog/2021/06/twitter-sentiment-analysis-a-nlp-use-case-for-beginners/

In [2]:
df = pd.read_csv('twitterdata.csv', encoding = "ISO-8859-1")
print('Loaded file.')
df = pd.DataFrame(np.vstack([df.columns, df])) # Moves column names into row 1
df.columns = ['sentiment', 'id', 'date', 'query', 'user', 'tweet'] # Renames columns
df.replace({'sentiment': {4: 1}}, inplace=True) # Replaces all '4's with '1's in column 'sentiment'
df.replace({'sentiment': {0: -1}}, inplace=True) # Replaces all '0's with '-1's in column 'sentiment'
data_types_dict = {
    'sentiment': int,
    'id': float
}
print('Fixed sentiment values.')
# df['unix_time'] = pd.to_datetime(df['date'], format='%a %b %d %H:%M:%S %Z %Y', errors='coerce').astype(int) / 10**9
# df.dropna(subset=['unix_time'], inplace=True)

# df['unix_time'] = df['date'].apply(lambda x: parser.parse(x, tzinfos={"PDT": -7*3600}).timestamp())
# print('Added unix column.')

df = df.astype(data_types_dict)
df.dtypes

Loaded file.
Fixed sentiment values.


sentiment      int32
id           float64
date          object
query         object
user          object
tweet         object
dtype: object

In [3]:
# df = pd.read_csv('twitterdata_unix.csv', encoding = "ISO-8859-1")
# df = df[['sentiment', 'tweet', 'unix_time']]
df = df.sample(frac=1).reset_index(drop=True) # shuffles rows so its not all 1 and then -1
df = df[['sentiment', 'tweet']]
df.head()

,sentiment,tweet
0,-1,omg. i just spent the whole day reading and do...
1,1,Twitter Wars: The New Hope Now I just have to...
2,1,"@DonnieWahlberg I have seen it, BOY you ROCK t..."
3,-1,Today is a make-up day I fear no spots but pa...
4,-1,my accomplishment of rationality and reason: m...


In [4]:
s = df.stack().value_counts()
print(s)

1                                                                                                 800000
-1                                                                                                799999
isPlayer Has Died! Sorry                                                                             210
good morning                                                                                         118
headache                                                                                             115
                                                                                                   ...  
@codizzle009 wooow we are both awake  cool!                                                            1
@mitsuba3 ã??ã?ã?ã?ã?²ã?¨ã?«ã?¯ã?@buzzter .  @p2pquak ã?£ã?¦ã?®ãã?ãã?ã?ã?ãï¼ï¼         1
i'm sad that the cute boy was only here to try and sell me trackpants                                  1
@mello826BA i never get my daily scope                 

In [5]:
# for i in range(10):
#     print(df['tweet'][i])
content = df['tweet'][:1000].values.tolist()
content = [x.strip() for x in content] # Deletes white space before and after
content[:20]


['omg. i just spent the whole day reading and doing this stupid packet.',
 'Twitter Wars: The New Hope  Now I just have to figure out which mac app is the rebel/imperial guard.',
 '@DonnieWahlberg I have seen it, BOY you ROCK that LOGO out',
 'Today is a make-up day I fear  no spots but patchy skin',
 "my accomplishment of rationality and reason: my mom thinks it was the biggest mistake of my life and STILL won't drop it, how frustrating",
 "I'm not going to be a bitch, so I won't be choked by Celia after Dani writes a news article about Twitter",
 '@Franklero haha ahhh frank you crazy kid',
 'Night night all, I can see more Cluedo tomorrow  shall I just let them win so I can get some PEACE',
 '@ChaosMagick Really?! Oh, OK then, just for you   http://twitpic.com/7gk5o',
 'Peeing while holding a squirmy baby is NOT an easy task!!! To make matters worse, Zoe was in the stall next 2 me pooping...    Ugh!!!',
 "@phete I just tried in 0.5 experimental and framework 'webkit' works fine. I do

## stop words #1

In [6]:
def full_remove(x, removal_list):
    for w in removal_list:
        x = x.replace(w, ' ')
    return x

## Remove digits
digits = [str(x) for x in range(10)]
remove_digits = [full_remove(x, digits) for x in content]

## Remove punctuation
remove_punc = [full_remove(x, list(string.punctuation)) for x in remove_digits]

## Make everything lower-case and remove any white space
sents_lower = [x.lower() for x in remove_punc]
sents_lower = [x.strip() for x in sents_lower]

## Remove stop words
from nltk.corpus import stopwords
stops = stopwords.words("English")
def removeStopWords(stopWords, txt):
    newtxt = ' '.join([word for word in txt.split() if word not in stopWords])
    return newtxt
content1 = [removeStopWords(stops,x) for x in sents_lower]
content1[:20]

['omg spent whole day reading stupid packet',
 'twitter wars new hope figure mac app rebel imperial guard',
 'donniewahlberg seen boy rock logo',
 'today make day fear spots patchy skin',
 'accomplishment rationality reason mom thinks biggest mistake life still drop frustrating',
 'going bitch choked celia dani writes news article twitter',
 'franklero haha ahhh frank crazy kid',
 'night night see cluedo tomorrow shall let win get peace',
 'chaosmagick really oh ok http twitpic com gk',
 'peeing holding squirmy baby easy task make matters worse zoe stall next pooping ugh',
 'phete tried experimental framework webkit works fine installed tho',
 'well looks like gonna rain screen green dinner besties instead',
 'rachecullen sucks',
 'rah rah want fine selection biscuits',
 'company party tonight fun hope somebody brings beer bong',
 'cuddles baby girl boyfriend watched play ball fun port hot even night wanna lake',
 'jojo yay read pool wasnt done yet poo lol',
 'excited working tomorrow 

## stops we defined instead

In [7]:
stop_set = ['the', 'a', 'an', 'i', 'he', 'she', 'they', 'to', 'of', 'it', 'from']
content2 = [removeStopWords(stop_set,x) for x in sents_lower]
content2[:20]

['omg just spent whole day reading and doing this stupid packet',
 'twitter wars new hope now just have figure out which mac app is rebel imperial guard',
 'donniewahlberg have seen boy you rock that logo out',
 'today is make up day fear no spots but patchy skin',
 'my accomplishment rationality and reason my mom thinks was biggest mistake my life and still won t drop how frustrating',
 'm not going be bitch so won t be choked by celia after dani writes news article about twitter',
 'franklero haha ahhh frank you crazy kid',
 'night night all can see more cluedo tomorrow shall just let them win so can get some peace',
 'chaosmagick really oh ok then just for you http twitpic com gk o',
 'peeing while holding squirmy baby is not easy task make matters worse zoe was in stall next me pooping ugh',
 'phete just tried in experimental and framework webkit works fine don t have installed tho',
 'well looks like s gonna rain no screen on green dinner with besties instead',
 'rachecullen sucks

## stemming

In [8]:
import nltk
def stemporter(words):
    porter = nltk.PorterStemmer()
    new_words = [porter.stem(w) for w in words]
    return new_words
    
def stemlancaster(words):
    porter = nltk.LancasterStemmer()
    new_words = [porter.stem(w) for w in words]
    return new_words    

porter = [stemporter(x.split()) for x in content2]
porter = [" ".join(i) for i in porter]
porter[0:10]

['omg just spent whole day read and do thi stupid packet',
 'twitter war new hope now just have figur out which mac app is rebel imperi guard',
 'donniewahlberg have seen boy you rock that logo out',
 'today is make up day fear no spot but patchi skin',
 'my accomplish ration and reason my mom think wa biggest mistak my life and still won t drop how frustrat',
 'm not go be bitch so won t be choke by celia after dani write news articl about twitter',
 'franklero haha ahhh frank you crazi kid',
 'night night all can see more cluedo tomorrow shall just let them win so can get some peac',
 'chaosmagick realli oh ok then just for you http twitpic com gk o',
 'pee while hold squirmi babi is not easi task make matter wors zoe wa in stall next me poop ugh']

## vectorizing

In [9]:
vectorizer = CountVectorizer(analyzer = "word", 
                             preprocessor = None, 
                             stop_words =  'english', 
                             max_features = 6000, ngram_range=(1,5))
data_features = vectorizer.fit_transform(sents_processed)
tfidf_transformer = TfidfTransformer()
data_features_tfidf = tfidf_transformer.fit_transform(data_features)
data_mat = data_features_tfidf.toarray()

NameError: name 'sents_processed' is not defined